In [ ]:
import pandas as pd
import numpy as np
from datetime import date
import requests
from bs4 import BeautifulSoup
import unicodedata
import time

In [ ]:
# creating url iterator 
start_date = date(2019, 1, 1)
end_date = date(2020, 1, 1)
daterange = pd.date_range(start_date, end_date)

dates = pd.DataFrame({"dates":daterange})
dates = dates.iloc[::5, :].reset_index(drop=True)

after = dates.iloc[::2].reset_index(drop=True)  # even
before = dates.iloc[1::2].reset_index(drop=True)  # odd

# the way their html functions requires overlapping date sets
after_offset = after + pd.DateOffset(1)
before_offset = before + pd.DateOffset(-1)

# format
after = after.dates.dt.strftime('%Y-%m-%d')
before = before.dates.dt.strftime('%Y-%m-%d')
after_offset = after_offset.dates.dt.strftime('%Y-%m-%d')
before_offset = before_offset.dates.dt.strftime('%Y-%m-%d')

# create date list
urls = []
for j in range(0,36):
  url_1 = "https://union.barstoolsports.com/stories/latest?limit=1500&after="+ after[j] + "&before=" + before[j] +"&type=standard_post"
  urls.append(url_1)
  url_2 = "https://union.barstoolsports.com/stories/latest?limit=1500&after="+ before_offset[j] + "&before=" + after_offset[j+1] +"&type=standard_post"
  urls.append(url_2)

# empty dataframe to append
Barstool = pd.DataFrame({
        "dates":[] ,
        "author": [],
        "title": [],
        "text": [],
        "comments": [],
        "blockquotes": [],
        "twitter": [],
        "instagram": [],
        "tiktok": [],
        "youtube": [],
        "images": [],
        "streamable": [],
        'gifs': []
        })



In [ ]:
for i in range(len(urls)):
  # read in directory
  info = pd.read_json(urls[i])['url']
  print("---", i, "---")
  # each directory has hundreds of blogs in chronological order
  for k in range(len(info)):
    try:
      print("---", k,"---")
      url = info[k]
      page = requests.get(url)
      soup = BeautifulSoup(page.content,"html.parser")
      words = [t.get_text() for t in soup.select(".story__content")][0]
      words = unicodedata.normalize('NFKD',words)
      author = [w.get_text() for w in soup.select(".authorName")][0]
      date = [d.get_text() for d in soup.select(".timestamp")][0]
      title = [p.get_text() for p in soup.select(".story__title")][0]
      comments = len(soup.find_all("span", { "class":"jsx-3955762224" }))
      links = soup.select('.iframely-embed')
      links = [i.findChild("a")['href'] for i in links]
      twitter = len([s for s in links if "twitter.com" in s])
      insta = len([s for s in links if "instagram.com" in s])
      tiktok = len([s for s in links if "tiktok.com" in s])
      streamable = len([s for s in links if "streamable.com" in s])
      youtube = len([s for s in links if "youtube.com" in s])
      imgs = len(soup.findAll("img", {"class": "fr-fic fr-dib bs-image"}))
      imgs = imgs + len(soup.findAll("img",{'class':'alignnone'}))
      blockquotes = len(soup.select('blockquote'))
      gifs = len(soup.find_all("video"))

      frame = pd.DataFrame({
            "dates": date,
            "author": author,
            "title": title,
            "text": words,
            "comments": [comments],
            "blockquotes": [blockquotes],
            "twitter": [twitter],
            "instagram": [insta],
            "tiktok": [tiktok],
            "youtube": [youtube],
            "images": [imgs],
            "streamable": [streamable],
            'gifs': [gifs]
            })
    
      Barstool = Barstool.append(frame)
    except:
      print("Barstool Booted me")
      print("Or some other error")
      print("Waiting 30 seconds...")
      time.sleep(30)
      continue


In [ ]:
!pip install xlsxwriter

     |████████████████████████████████| 153kB 13.6MB/s 


In [ ]:
import pickle
with open('./Barstool_2019.pkl', 'wb') as fid:
     pickle.dump(Barstool, fid)

In [ ]:
Barstool.to_excel('Barstool_2019.xlsx', engine='xlsxwriter')